# Hyper parameter tunning
- updated 0916
- data  
5th_FE_ver4_before   
5th_FE_ver4  

- catboost 에만 label encoding data 사용한다.

- catboost 튜닝

## y_predict 값들 피클로 저장해놓기

## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options
Selecting previously unselected package fonts-nanum.
(Reading database ... 144600 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig 

In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

## Load Data
- 5th_FE_ver4_before  
- 5th_FE_ver4


data['X'] / data['y']

### 5th_FE_ver4

In [ ]:
# load data
## ver3
v4 = joblib.load(os.path.join('data', '5th_FE_ver4.pkl'))
locals().update(v4)

v4_label = joblib.load(os.path.join('data', '5th_FE_ver4_before.pkl'))
locals().update(v4_label)

In [ ]:
# X, y 설정
X = v4['X'] ; y = v4['y'] ; log_y = np.log(y)
X_label = v4_label['X']

In [ ]:
X.isnull().sum().sum(), X_label.isnull().sum().sum()

(267028, 267028)

In [ ]:
# fill na
X = X.fillna(0); X_label = X_label.fillna(0)

# One Hot
X_dum = pd.get_dummies(X)
X.shape, X_dum.shape, X_label.shape

((35379, 334), (35379, 334), (35379, 276))

### Cross Validation function

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

In [ ]:
# MAPE_exp_cv 구현
def MAPE_exp_cv5(model,X,y,cv_splits):        
    # setting
    mape = []
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=42,shuffle=True)
    for t,v in cv.split(X):
        X_train=X.iloc[t]       # 훈련용
        X_val=X.iloc[v]         # 검증용 분리.
        y_train=y.iloc[t]
        y_val=y.iloc[v]

        # model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        #y_pred_list.append(np.exp(y_pred))
        # mape
        mape.append(MAPE_exp(y_val, y_pred))
    return mape, np.mean(mape), y_pred_list

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_cv5_st(model,X,y,cv_splits):        
    # setting
    mape = []
    pred_list = []     # X_val_ 으로 예측
    test_list = []     # X_test 으로 예측
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        # model
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred_list.append(np.exp(y_pred))
        
        test_pred = model.predict(X_test)
        test_list.append(np.exp(test_pred))
        # mape
        mape.append(MAPE_exp(y_val_, y_pred))
    
    # save
    
    return mape, np.mean(mape), pred_list, test_list 

### Modeling with cv

### CatBoost
예측해야하는 값이 훈련 셋 피처에 들어가버리는 문제, 즉 Data Leakage 문제  
doc : https://catboost.ai/docs/concepts/python-reference_utils_eval_metric.html  
kaggle : https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview
  
    
mape : 31.306861591528538

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 65kB/s 


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
# catboost setting
cb_model = CatBoostRegressor(iterations=5000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = 0.2,
                             early_stopping_rounds = 500,
                             #group_id = []
                             od_type='Iter',
                             metric_period = 50,     # ?
                             #od_wait=20,
                             thread_count=-1)

평균 mape 32.73414280484671  
arraty 5개씩 * 2 개 잘나옴

### tunning

In [ ]:
categorical_features_indices = np.where(X_label.dtypes != np.float)[0];categorical_features_indices

array([  1,   2,   3,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
        17,  29,  30,  32, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169])

In [ ]:
np.where(X_dum.dtypes != np.float)[0];categorical_features_indices

array([  1,   5,   6,   7,   8,   9,  22, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 266, 267,
       268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293,
       294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306,
       307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319,
       320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332,
       333])

### catboost + gridsearch
label data 에서 Bad value for num_feature 이슈때문에 X_dum 사용  
***
{'params': {'bagging_temperature': 86.20566967662191,  
  'border_count': 8.920475731018419,  
  'depth': 8.834206374187707,  
  'learning_rate': 0.012401575319148032,  
  'min_data_in_leaf': 5.817638932445064},  
 'target': -28.91074826341997}  

In [ ]:
# Setting X, y
X = X_dum
y = log_y

# params
param_grid = {
'learning_rate':[0.01,0.1,0.05],
'min_data_in_leaf': range(10,20,2),
'depth': range(5,30,5),
'border_count': range(10,300,20),
'bagging_temperature' : range(1,1000,100)      # baysian bootstrap
}

params = {}
params['iterations'] = 5000
#params['custom_loss'] = 'MAPE'
params['eval_metric'] = 'MAPE'
params['random_seed'] = 77
# 튜닝 내용
#params['learning_rate'] = learning_rate
# params['min_data_in_leaf'] = int(round(min_data_in_leaf))    
# params['depth'] = int(round(depth))         
# #params['max_leaves'] = int(round(max_leaves))
# #params['l2_leaf_reg'] = int(round(l2_leaf_reg))
# params['border_count'] = int(round(border_count))
# params['bagging_temperature'] = int(round(bagging_temperature))

# model
cb_model = CatBoostRegressor(**params ,
                             early_stopping_rounds = 500,
                             #task_type = "GPU",
                             leaf_estimation_iterations = 10,
                             #use_best_model=True,
                             od_type="Iter",
                             #logging_level='Silent',
    )
    
# fit
cb_model.fit(X, y, 
             #cat_features=categorical_features_indices,
             #eval_set=(X_val_, y_val_), 
             verbose = 1000)

# param_grid = {
# 'learning_rate':[0.01,0.1,0.05],
# 'min_data_in_leaf': range(10,20,2),
# 'depth': range(5,30,2),
# 'border_count': [0.1,0.2,0.3,0.4,0.5,0.6],
# 'baggiing_temperature' : [0.1,0.2,0.3,0.5]
# }


# define grid_search
grid_search = GridSearchCV(estimator=cb_model, 
                        param_grid=param_grid,  
                        n_jobs=-1, 
                        verbose=2
                        )

# fit with (x_train, y_train)
grid_search.fit(X, y)

0:	learn: 0.0486700	total: 70.5ms	remaining: 5m 52s
1000:	learn: 0.0194718	total: 22.6s	remaining: 1m 30s
2000:	learn: 0.0166885	total: 45.5s	remaining: 1m 8s
3000:	learn: 0.0151143	total: 1m 8s	remaining: 45.5s
4000:	learn: 0.0139492	total: 1m 31s	remaining: 22.9s
4999:	learn: 0.0130280	total: 1m 54s	remaining: 0us
Fitting 5 folds for each of 11250 candidates, totalling 56250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


In [ ]:
# best params
grid_search.best_params_